In [1]:
import sqlite3
db = sqlite3.connect('./database')
db.isolation_level = None
cursor = db.cursor()

In [259]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS gene_features_all(id TEXT, trans_id TEXT, prot_id TEXT, chrom INTEGER, start INTEGER, end INTEGER, gLength INTEGER, cds INTEGER, name TEXT, exp REAL, ess REAL, evolRate REAL, dup_type TEXT)
''')
with open('lens_locs_2.txt') as source:
    source.readline()
    for line in source:
        line = line.strip().split('\t')
        gene_id, transcript_id, prot_id, chrom, start, end, CDS, name = line
        gLen = (abs(int(end)-int(start))) +1 #start and end bases are both within the gene
        cursor.execute(
                '''INSERT INTO gene_features_all(id, trans_id, prot_id, chrom, start, end, gLength, cds, name) VALUES(?,?,?,?,?,?,?,?,?)''', 
                (gene_id, transcript_id, prot_id, chrom, start, end, gLen, CDS, name))
        db.commit()

In [260]:
#essentiality
with open('Downloads/Wang_CRISPR.csv', 'r') as file:
    file.readline()
    for line in file:
        l = line.strip().split(',')
        geneName, KBM7, K562, Jiyoye, Raji = l[0], float(l[2]), float(l[4]), float(l[6]), float(l[8])
        scoreList = [KBM7, K562, Jiyoye, Raji]
        ess = min(scoreList)
        cursor.execute('''UPDATE gene_features_all SET ess = ? WHERE name == ?''', 
                       (ess, geneName))
        db.commit()

In [261]:
#evolutionary rate
count = 0
with open('Downloads/dn_ds_chimp.txt','r') as evol_file_chimp:
    evol_file_chimp.readline()
    for line in evol_file_chimp:
        count += 1
        line = line.strip().split('\t')
        if len(line) == 4:
            gene_id_ev, trans_id_ev, dn, ds = line
            if float(ds) == 0:
                rate = 0
            else:
                rate = float(dn)/float(ds)
        else:
            gene_id_ev, trans_id_ev = line
            dn = '-'
            ds = '-'
            rate = '-'
        cursor.execute('''UPDATE gene_features_all SET evolRate = ? WHERE 
                        (id == ?)''', 
                       (rate, gene_id_ev))
    db.commit()

In [ ]:
#id to tissue
cursor.execute('CREATE TABLE IF NOT EXISTS sample_att(Gtex_id TEXT, tissue TEXT)')
with open('Downloads/GTex_sample_data.txt') as sample_att_file:
    sample_att_file.readline()
    for line in sample_att_file:
        line = line.strip().split('\t')
        GTex_id, tissue = line[0], line[5]
        cursor.execute('INSERT INTO sample_att(Gtex_id, tissue) VALUES(?,?)', (GTex_id, tissue))
    db.commit()

In [29]:
#gene expression
count = 0
pcount = 0
import re
import numpy as np

cursor.execute('CREATE TABLE IF NOT EXISTS Gene_expression_data(id TEXT, name TEXT, tissue TEXT, avg_exp REAL)')

with open('Downloads/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.txt') as expression_file:
    for i in range(2):
        expression_file.readline()
    sample_ids = expression_file.readline().strip().split('\t')[2:]
    #gets where each tissue starts and ends in the sample ids
    tissue_index = {}
    old_tissue = ''
    for i in range(0,len(sample_ids)):
        cursor.execute('''SELECT tissue FROM sample_att WHERE GTex_id == ?''', (sample_ids[i],))
        tissue = cursor.fetchall()[0][0]
        if tissue == old_tissue:
            tissue_index[tissue].append(i)
        else:
            tissue_index[tissue] = [i]
            old_tissue = tissue
    #each line is a gene
    print('starting')
    for line in expression_file:
        count += 1
        av_dict = {}
        count += 1
        line = line.strip().split('\t')
        #gives the gene id without the version number
        gene = re.search(r'(.*)\.(.*)', line[0]).group(1)
        name = line[1]
        exp_values = line[2:]
        for i in range(0,len(exp_values)):
             exp_values[i] = float(exp_values[i])
        for tissue in tissue_index:
            mini = min(tissue_index[tissue])
            maxi = max(tissue_index[tissue]) + 1
            #get the average expression for each tissue
            av_dict[tissue] = np.median(exp_values[mini:maxi])
            cursor.execute('''INSERT INTO Gene_expression_data(id, name, tissue, avg_exp) VALUES (?,?,?,?)''',
                          (gene, name, tissue, av_dict[tissue]))
        if count % 2810 == 0:
            pcount += 5
            print(str(pcount) + '%')
        #need to add the expression data from other sources 
db.commit()

starting


OperationalError: near "(": syntax error

In [ ]:
#transcript expression
count = 0
pcount = 0
cursor.execute('CREATE TABLE IF NOT EXISTS Transcript_expression_data(id TEXT, trans_id TEXT, tissue TEXT, avg_exp REAL)')

with open('Downloads/GTEx_Analysis_2016-01-15_v7_RSEMv1.2.22_transcript_tpm.txt') as t_exp_file:
    #the first two items aren't ids
    sample_ids_2 = t_exp_file.readline().strip().split('\t')[2:]
    tissue_index = {}
    old_tissue = ''
    for i in range(0,len(sample_ids_2)):
        cursor.execute('''SELECT tissue FROM sample_att WHERE GTex_id == ?''', (sample_ids[i],))
        tissue = cursor.fetchall()[0][0]
        if tissue == old_tissue:
            tissue_index[tissue].append(i)
        else:
            tissue_index[tissue] = [i]
            old_tissue = tissue
    for line in t_exp_file:
        av_dict = {}
        count += 1
        line = line.strip().split('\t')        
        transcript = re.search(r'(.*)\.(.*)', line[0]).group(1)
        gene = re.search(r'(.*)\.(.*)', line[1]).group(1)
        t_exp_values = line[2:]
        for i in range(0,len(t_exp_values)):
             t_exp_values[i] = float(t_exp_values[i])
        for tissue in tissue_index:
            mini = min(tissue_index[tissue])
            maxi = max(tissue_index[tissue]) + 1
            #get the average expression for each tissue
            av_dict[tissue] = np.mean(t_exp_values[mini:maxi])
            cursor.execute('''INSERT INTO Transcript_expression_data(id, trans_id, tissue, avg_exp) VALUES (?,?,?,?)''',
                          (gene, transcript, tissue, av_dict[tissue])) 
        if count%9826 == 0:
            pcount += 5
            print(str(pcount) + '%')
db.commit()

In [281]:
cursor.execute('''SELECT COUNT(DISTINCT id) FROM gene_features_all WHERE (evolRate == 0 OR evolRate == "-")''')
print(cursor.fetchall())

[(5949,)]


In [270]:
from statistics import mode, StatisticsError
import time
cursor.execute('''CREATE TABLE IF NOT EXISTS gene_features(id TEXT, chrom INTEGER, start INTEGER, end INTEGER, 
                gLength INTEGER, cds_len INTEGER, cds_exp INTEGER, name TEXT, 
                exp REAL, ess REAL, evolRate REAL, dup_type TEXT, dev TEXT)''')
cursor.execute('''SELECT DISTINCT id FROM gene_features_all''')
count = 0
pcount = 0
id_list = []
[id_list.append(y[0]) for y in cursor.fetchall()]
query_list = []
print('starting')
t1 = time.time()
for gene_id in id_list:
    count += 1
    row_list = []
    cursor.execute('''SELECT * FROM gene_features_all WHERE id == ?''', (gene_id,))
    [row_list.append(x) for x in cursor.fetchall()]
    row1 = row_list[0]
    i,c,s,e,gL,n,es,eR,dT = row1[0], row1[3], row1[4], row1[5], row1[6], row1[8], row1[10], row1[11], row1[12]
    #cds by length
    cds_list = []
    [cds_list.append(row[7]) for row in row_list]
    cds = max(cds_list)
    cursor.execute('''SELECT avg_exp FROM Gene_expression_data WHERE id == ?''', (gene_id,))
    exp_list = []
    [exp_list.append(x[0]) for x in cursor.fetchall()]
    try:
        ex = max(exp_list)
    except ValueError:
        ex = -1
    #CDS by expression
    tList = []
    cursor.execute('''SELECT id, tissue, trans_id, MAX(avg_exp) 
                    FROM Transcript_expression_data
                    WHERE id = ?
                    GROUP BY id,tissue''',
                    (gene_id,))
    [tList.append(y[2]) for y in cursor.fetchall()]
    try:
        tt = mode(tList)
        cursor.execute('''SELECT cds FROM gene_features_all WHERE trans_id = ?''', (tt,))
        cds_e = cursor.fetchall()[0][0]
    except (IndexError, StatisticsError) as error:
        cds_e = -1
    query_list.append((i,c,s,e,gL,cds,cds_e,n,ex,es,eR,dT))
    if (count%250 == 0) or (count == len(id_list)):
        cursor.executemany('''INSERT INTO gene_features(id,chrom,start,end,gLength,cds_len,cds_exp,name,exp,ess,evolRate,dup_type) 
                    VALUES (?,?,?,?,?,?,?,?,?,?,?,?)''', query_list)
        query_list.clear()
        print(time.time()-t1)
    if count%1014 == 0:
        pcount += 5
        print(str(pcount) + '%' + ' done')
db.commit()

starting
635.4381759166718
1262.0247631072998
1884.391834974289
2505.430620908737
5% done
3124.3054060935974
3742.8106050491333
4361.413789987564
4979.523952960968
10% done
5621.419725894928
6321.52587389946
7014.365797996521
7715.520747900009
15% done
8397.515458106995
9095.19866490364
9793.084975004196
10447.31580901146
20% done
11076.027450084686
11703.624747037888
12331.253510951996
12959.032908916473
25% done
13585.660911083221
14212.110119104385
14838.145415067673
15463.204093933105
30% done
16088.090156078339
16712.52544593811
17336.91012096405
17964.470082998276
35% done
18600.17510008812
19242.171489953995
19877.183581113815
20542.92020010948
40% done
21156.398781061172
21785.615838050842
22448.54932999611
23090.75127005577
45% done
23723.3496260643
24332.68085002899
24935.235861063004
25536.698832035065
50% done
26137.39456510544
26737.58636689186
27336.265707969666
27933.643132925034
55% done
28530.880835056305
29127.561028003693
29723.399254083633
30319.195180892944
60% don

In [272]:
cursor.execute('''ALTER TABLE gene_features ADD COLUMN dev_exp REAL''')

In [273]:
# updating exp column with brain dev data
count = 0
qList = []
t1 = time.time()
with open('Downloads/brain_dev_exp.tsv', 'r') as brain_file:
    for i in range(0,4):
        brain_file.readline()
    tissues = brain_file.readline().strip().split('\t')[2:]
    old_max = 0
    for line in brain_file:
        count += 1
        values = []
        line = line.strip().split('\t')
        gene_id = line[0]
        name = line[1]
        for x in line[2:]:
            try:
                values.append(float(x)) 
            except ValueError:
                [values.append(None)]
        max_dev = max(x for x in values if x is not None)
        qList.append((max_dev, gene_id))
        if count % 500 == 0:
            print(count)
            print((time.time()-t1)/60)
            t1 = time.time()
            cursor.executemany('''UPDATE gene_features SET dev_exp = ? 
                                WHERE (id == ?) ''', qList)
            qList.clear()
    cursor.executemany('''UPDATE gene_features SET dev_exp = ? 
                        WHERE (id == ?)''', qList)
db.commit()

500
0.0011044661204020181
1000
0.05184411605199178
1500
0.048823233445485434
2000
0.048802681763966876
2500
0.047989384333292646
3000
0.047914516925811765
3500
0.04811926682790121
4000
0.04789576530456543
4500
0.047833800315856934
5000
0.04774726629257202
5500
0.04774381717046102
6000
0.048572699228922524
6500
0.04834776719411214
7000
0.04783725341161092
7500
0.04757406711578369
8000
0.04847033421198527
8500
0.048020180066426596
9000
0.04819468259811401
9500
0.04792136748631795
10000
0.04921886920928955
10500
0.04798630078633626
11000
0.04842958052953084
11500
0.04829626480738322
12000
0.048191781838734946
12500
0.04849486748377482
13000
0.047908631960550944
13500
0.04809266726175944
14000
0.04816731611887614
14500
0.047990016142527264
15000
0.04824965000152588
15500
0.04818271398544312
16000
0.04872804880142212
16500
0.048387197653452556
17000
0.051161948839823404
17500
0.04892956813176473
18000
0.04781428575515747
18500
0.0482727845509847
19000
0.048763434092203774
19500
0.0489410360

In [274]:
#fetal data (FANTOM5)
i_list = []
values = []
q_list = []
count = 0
with open('Downloads/fetal_exp.tsv', 'r') as ffile:
    for i in range(0,4):
        ffile.readline()
    header = ffile.readline().strip().split('\t')
    for i in range(0, len(header)):
        if 'fetal' in header[i]:
             i_list.append(i)
    for line in ffile:
        count += 1
        values = []
        line = line.strip().split('\t')
        gene_id = line[0]
        for n in i_list:
            try:
                values.append(line[n])
            except IndexError:
                values.append(None)
        max_val = max(x for x in values if x is not None)
        q_list.append((max_val, gene_id, max_val))
        if count % 500 == 0:
            cursor.executemany('''UPDATE gene_features 
                                SET dev_exp = ? 
                                WHERE (id ==?) AND (dev_exp < ?)''', q_list)
            q_list.clear()
            print(count)
    cursor.executemany('''UPDATE gene_features 
                        SET dev_exp = ? 
                        WHERE (id ==?) AND (dev_exp < ?)''', q_list) 

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000


In [49]:
cursor.execute('''DROP TABLE ohnologs''')
cursor.execute('''CREATE TABLE IF NOT EXISTS ohnologs(id TEXT, ohnos TEXT)''')
o_dict = {}
with open('Desktop/human_ohnologs_EnsV86.txt', 'r') as o_file:
    for line in o_file:
        line = line.strip().split('\t')
        gene, ohno = line[0], line[1]
        if gene in o_dict.keys():
            o_dict[gene].append(ohno)
        else:
            o_dict[gene] = [ohno]
for key in o_dict:
    ohnos = ' '.join(o_dict[key])
    cursor.execute('INSERT INTO ohnologs(id,ohnos) VALUES(?,?)', (key,ohnos))
# for key in o_dict:
#     try:
#         cursor.execute('SELECT id FROM gene_features_all WHERE prot_id == ?', (key,))
#         gene = cursor.fetchall()[0][0]
#     except IndexError:
#         next
#     cursor.execute('SELECT ohnos FROM ohnologs WHERE id ==?', (key,))
#     no = len(cursor.fetchall()[0][0].split(' '))
#     if no > 4:
#         print(gene, key, ' WGD:', no)

In [85]:
cursor.execute('''DROP TABLE paralogs''')
cursor.execute('''CREATE TABLE IF NOT EXISTS paralogs(id TEXT, prot TEXT, paralogs TEXT, paralogs_p TEXT)''')
pDictG = {}
pDictP = {}
cDict = {}
with open('Downloads/paralogs2.txt','r') as pFile:
    pFile.readline()
    for line in pFile:
        line = line.strip().split('\t')
        gene_id, prot_id, para_g, para_p = line[0], line[1], line[2], line[3]
        if gene_id in pDictG.keys():
            if prot_id not in cDict[gene_id]:
                cDict[gene_id].append(prot_id)
            if para_g not in pDictG[gene_id]:
                pDictG[gene_id].append(para_g)
            if para_p not in pDictP[gene_id]:
                pDictP[gene_id].append(para_p)
        else:
            cDict[gene_id] = [prot_id]
            pDictG[gene_id] = [para_g]
            pDictP[gene_id] = [para_p]
    for key in pDictG:
        paraGs = ' '.join(pDictG[key])
        paraPs = ' '.join(pDictP[key])
        prots = ' '.join(cDict[key])
        cursor.execute('''INSERT INTO paralogs(id, prot, paralogs, paralogs_p) VALUES (?,?,?,?)''', 
                       (key, prots, paraGs, paraPs))

In [50]:
cursor.execute('''ALTER TABLE ohnologs ADD COLUMN g_id TEXT''')
cursor.execute('''UPDATE ohnologs SET g_id = 
                (SELECT id FROM gene_features_all WHERE prot_id == ohnologs.id)''')

In [133]:
cursor.execute('UPDATE gene_features SET dup_type = "singleton" WHERE id NOT IN (SELECT id FROM paralogs_id)')
ids = []
cursor.execute('''SELECT id FROM gene_features WHERE dup_type != "singleton"''')
[ids.append(x[0]) for x in cursor.fetchall()]
count = 0
count_d = 0
count_check = 0
for i in ids:
    WGD = 'No'
    SSD = 'No'
    para_count = 0
    WGD_count = 0
    SSD_count = 0
    cursor.execute('SELECT paralogs_p FROM paralogs_id WHERE id ==?', (i,))
    paras = cursor.fetchall()[0][0].split(' ')
    cursor.execute('SELECT ohnos FROM ohnologs WHERE g_id == ?', (i,))
    try:
        ohnos = cursor.fetchall()[0][0].split(' ')
        for p in paras:
            para_count += 1
            if p in ohnos:
                WGD = 'Yes'
                WGD_count += 1
            elif p not in ohnos:
                SSD = 'Yes'
                SSD_count += 1
    except IndexError:
        WGD = 'No'
        SSD = 'Yes'
    if (WGD == 'Yes') and (SSD == 'Yes'):
        count += 1
        cursor.execute('''UPDATE gene_features SET dup_type = "WGD/SSD" WHERE id ==?''', (i,))
    elif WGD == 'Yes':
        cursor.execute('''UPDATE gene_features SET dup_type = "WGD" WHERE id == ?''', (i,))
    elif SSD == 'Yes':
        cursor.execute('''UPDATE gene_features SET dup_type = "SSD" WHERE id == ?''', (i,))
    else:
        print(i)
        print('Problem!')
    count_check +=1
    if count_check % 500 == 0:
        print(count_check * 100 /len(ids))
db.commit()
#             print(i)
#             print(para_count, WGD_count, SSD_count)
#         if SSD_count == 1:
#             count_d += 1
    
    
# print(count)
# print(count_d)

5.963740458015267
11.927480916030534
17.891221374045802
23.85496183206107
29.818702290076335
35.782442748091604
41.74618320610687
47.70992366412214
53.6736641221374
59.63740458015267
65.60114503816794
71.56488549618321
77.52862595419847
83.49236641221374
89.45610687022901
95.41984732824427


In [316]:
cursor.execute('''SELECT  COUNT (DISTINCT id) FROM paralogs WHERE vertebrate = "Yes"''')
print(cursor.fetchall())

[(12164,)]


In [286]:
cursor.execute('''DROP TABLE paralogs_id''')

In [131]:
ids = []
cursor.execute('''SELECT id FROM gene_features WHERE id IN (SELECT id FROM paralogs_id)''')
[ids.append(x[0]) for x in cursor.fetchall()]
count = 0
both_count = 0
WGD_count = 0
SSD_count = 0
for i in ids:
    count += 1
    WGD = 'No'
    SSD = 'No'
    para_count = 0

    cursor.execute('SELECT paralogs_p FROM paralogs_id WHERE id ==?', (i,))
    paras = cursor.fetchall()[0][0].split(' ')    
    cursor.execute('SELECT ohnos FROM ohnologs WHERE g_id == ?', (i,))
    try:
        ohnos = cursor.fetchall()[0][0].split(' ')
        for p in paras:
            para_count += 1
            if p in ohnos:
                WGD = 'Yes'
            elif p not in ohnos:
                SSD = 'Yes'
    except IndexError:
        WGD = 'No'
        SSD = 'Yes'
    if (WGD == 'Yes') and (SSD == 'Yes'):
        both_count += 1
    elif WGD == 'Yes':
        WGD_count += 1
    elif SSD == 'Yes':
        SSD_count += 1
    if count % 500 == 0:
        print(count*100/14218)
print(both_count)
print(WGD_count)
print(SSD_count)

3.516669011112674
7.033338022225348
10.550007033338023
14.066676044450697
17.58334505556337
21.100014066676046
24.616683077788718
28.133352088901393
31.650021100014065
35.16669011112674
38.683359122239416
42.20002813335209
45.71669714446476
49.233366155577436
52.75003516669011
56.26670417780279
656
3159
4569


In [11]:
names = []
e_names = []
cursor.execute('''SELECT name FROM gene_features''')
[names.append(x[0]) for x in cursor.fetchall()]
with open('Downloads/Wang_CRISPR.csv', 'r') as file:
    file.readline()
    for line in file:
        l = line.strip().split(',')
        geneName = l[0]
        e_names.append(geneName)
for n in names:
    if n not in e_names:
        cursor.execute('''UPDATE gene_features SET ess = -10 WHERE name == ?''', (n,))


In [ ]:
#check differences in IDs across GrCH37/GrCH38
with open('Downloads/Results-Homo_sapiens_Tools_IDMapper_.csv', 'r') as file1:
    file1.readline()
    count = 0
    not_exists = []
    not_exists2 = []
    for line in file1:
        line = line.strip().split(',')
        if line[0] != line[1]:
            count += 1
            not_exists.append(line[0])
            not_exists2.append(line[1])
            print('found one!')
            print(line[0], line[1])
    print('Old IDs: ' + str(len(set(not_exists))))
    print('New IDs: ' + str(len(set(not_exists2))))
with open('Downloads/Results-Homo_sapiens_Tools_IDMapper_-2.csv', 'r') as file2:
    file2.readline()
    not_exists2 = []
    not_exists = []
    for line in file2:
        line = line.strip().split(',')
        if line[0] != line[1]:
            count += 1
            not_exists.append(line[0])
            not_exists2.append(line[1])
            print('found one!')
            print(line[0], line[1])
    print('Old IDs: ' + str(len(set(not_exists))))
    print('New IDs: ' + str(len(set(not_exists2))))

In [282]:
import json
import time
import requests
start_time = time.time()
cursor.execute('''DROP TABLE paralogs''')
cursor.execute('''CREATE TABLE IF NOT EXISTS paralogs(id TEXT, para TEXT, tax_group TEXT, vertebrate TEXT)''')
old_tax_list = ['Opisthokonta','Animalia','Bilateria', 'Chordata'] 
new_tax_list = ['Vertebrata', 'Sarcopterygii','Euteleostomi', 'Tetrapoda', 
                'Amniota', 'Mammalia', 'Theria', 'Eutheria','Boreoeutheria',  
                'Euarchontoglires', 'Primates', 'Haplorrhini', 'Simiiformes',
                'Catarrhini', 'Hominoidea', 'Hominidae','Homininae', 'Homo sapiens']
id_list = []
cursor.execute('''SELECT DISTINCT id FROM gene_features''')
[id_list.append(e[0]) for e in cursor.fetchall()]
t1 = time.time()
act_count = 0
act_time = time.time()
count = 0
arg_list = []

for i in id_list:
    f_count = 0
    act_count += 1
#     while True:
#         try:
    if (time.time() - t1 <= 1) and (count == 10):
        time.sleep(2)
        t1 = time.time()
        count = 0
        url = 'http://rest.ensembl.org/homology/id/' + i + '?type=paralogues;target_species=human;sequence=none;'
        headers = {'Content-type':'application/json'}
        res = requests.get(url, headers= headers).json()
        count += 1
        # 1 entry dictionary with key 'data' that has the value of a one entry list 
        # which is a dictionary with two keys, id (value is queried id) and homologies
        # if id has no paralogs top dict structure is: {"data":[{"homologies":[],"id":"ENSG00000174059"}]}
        if res['data'][0]['homologies'] != []:
            for y in res['data'][0]['homologies']:
                para = y['target']['id']
                tax = y['taxonomy_level']
                if tax not in new_tax_list:
                    v = 'No'
                else:
                    v = 'Yes'
                arg_list.append((i,para,tax,v))
        else:
            next
    else:
        url = 'http://rest.ensembl.org/homology/id/' + i + '?type=paralogues;target_species=human;sequence=none;'
        headers = {'Content-type':'application/json'}
        res = requests.get(url, headers= headers).json()
        count += 1
        if res['data'][0]['homologies'] != []:
            for y in res['data'][0]['homologies']:
                para = (y['target']['id'])
                tax = y['taxonomy_level']
                if tax not in new_tax_list:
                    v = 'No'
                else:
                    v = 'Yes'
                arg_list.append((i,para,tax,v))
        else:
            next

#         except KeyError:
#             f_count += 1
#             if f_count <= 5:
#                 continue
#             else:
#                 print(res)
#                 print(i, 'GAVE UP')
#                 break
        
    if act_count%200 == 0:
        print(act_count)
        print((time.time() - act_time)/60)
        act_time = time.time()
        cursor.executemany('''INSERT INTO paralogs(id,para,tax_group,vertebrate) VALUES (?,?,?,?)''', arg_list)
        arg_list = []
    elif act_count == len(id_list):
        cursor.executemany('''INSERT INTO paralogs(id,para,tax_group,vertebrate) VALUES (?,?,?,?)''', arg_list)
        print('DONE. It took:', (time.time()-start_time)/60, 'minutes')



200
1.513385033607483
400
1.3175157308578491
600
1.242250100771586
800
1.2137322187423707
1000
1.0613779862721762
1200
1.090402082602183
1400
1.0246300498644512
1600
1.0183123668034872
1800
0.889645266532898
2000
0.9068204998970032
2200
1.0222475369771322
2400
1.0800101002057394
2600
0.9798417329788208
2800
0.9599148511886597
3000
0.836577848593394
3200
0.8015906810760498
3400
0.8262151837348938
3600
0.7506655017534892
3800
0.7708566506703695
4000
0.7611650665601094
4200
0.7884288668632508
4400
0.7755329012870789
4600
0.7567903836568196
4800
0.7247156977653504
5000
0.725066598256429
5200
0.7604548533757528
5400
0.7855380813280741
5600
0.7024110833803813
5800
0.6524755835533143
6000
0.5947569847106934
6200
0.5775078018506368
6400
0.6204972704251607
6600
0.6386296470959981
6800
0.657722532749176
7000
0.5780431509017945
7200
0.5558906356493633
7400
0.5132234493891398
7600
0.5244932810465495
7800
0.5630014181137085
8000
0.5894444863001506
8200
0.5356696367263794
8400
0.5369086821873983
860

In [372]:
cursor.execute('''UPDATE gene_features SET dup_type = "-"''')
cursor.execute('''UPDATE gene_features SET dup_type = "singleton" WHERE id NOT IN 
                (SELECT id FROM paralogs WHERE vertebrate = "Yes")''')
ids = []

cursor.execute('''SELECT id FROM gene_features WHERE dup_type != "singleton"''')
[ids.append(x[0]) for x in cursor.fetchall()]

count = 0
count_d = 0
count_check = 0
for i in ids:
    WGD = 'No'
    SSD = 'No'
    paras = []
    ohnos = []
    cursor.execute('SELECT para FROM paralogs WHERE id ==? AND vertebrate = "Yes"', (i,))
    [paras.append(x[0]) for x in cursor.fetchall()]
    cursor.execute('SELECT ohno_id FROM ohnologs_gene_ids WHERE id == ? AND vertebrate = "Yes"', (i,))
    try:
        [ohnos.append(x[0]) for x in cursor.fetchall()]        
        for p in paras:
            if p in ohnos:
                WGD = 'Yes'
                WGD_count += 1
            elif p not in g_ohnos:
                SSD = 'Yes'
                SSD_count += 1
    except IndexError:
        WGD = 'No'
        SSD = 'Yes'
    if (WGD == 'Yes') and (SSD == 'Yes'):
        count += 1
        cursor.execute('''UPDATE gene_features SET dup_type = "WGD/SSD" WHERE id ==?''', (i,))
    elif WGD == 'Yes':
        cursor.execute('''UPDATE gene_features SET dup_type = "WGD" WHERE id == ?''', (i,))
    elif SSD == 'Yes':
        cursor.execute('''UPDATE gene_features SET dup_type = "SSD" WHERE id == ?''', (i,))
    else:
        print(i)
        print('Problem!')
    count_check +=1
    if count_check % 500 == 0:
        print(count_check * 100 /len(ids))

4.110489970404473
8.220979940808945
12.331469911213416
16.44195988161789
20.55244985202236
24.662939822426832
28.773429792831305
32.88391976323578
36.994409733640246
41.10489970404472
45.21538967444919
49.325879644853664
53.43636961525814
57.54685958566261
61.65734955606708
65.76783952647156
69.87832949687603
73.98881946728049
78.09930943768497
82.20979940808944
86.32028937849391
90.43077934889838
94.54126931930286
98.65175928970733


In [378]:
cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type = "singleton"''')
print('Singletons:', cursor.fetchall()[0][0])

cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type = "SSD"''')
print('Small scale duplicates:', cursor.fetchall()[0][0])

cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type = "WGD"''')
print('Ohnologs:', cursor.fetchall()[0][0])

cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type = "WGD/SSD"''')
print("Things that shouldn't be:", cursor.fetchall()[0][0])

Singletons: 8113
Small scale duplicates: 6115
Ohnologs: 4940
Things that shouldn't be: 1109


In [237]:
cursor.execute('''SELECT DISTINCT COUNT(id) FROM gene_features 
                    WHERE id IN (SELECT id FROM paralogs WHERE vertebrate = "No") 
                    AND dup_type = "singleton"''')
cursor.fetchall()

[(3703,)]

In [307]:
cursor.execute('''SELECT * FROM ohnologs''')
count = 0
g_ohnos = []
forList = cursor.fetchall()
for x in forList:
    count += 1
    f = x[2]
    ohnos = x[1].split(' ')
    for e in ohnos:
        cursor.execute('''SELECT id FROM gene_features_all WHERE prot_id = ?''', (e,))
        try:
            g_e = cursor.fetchall()[0][0]
        except IndexError:
            print('Protein not found:', e)
        cursor.execute('''SELECT * FROM paralogs WHERE id = ? AND para = ? AND vertebrate = "No"''', (f,e))
        early_ohno_pairs = cursor.fetchall()
        if early_ohno_pairs != []:
            print(f, early_ohno_pairs)
    if count % 700 == 0:
        print((count/len(forList))*100, '% done')

Protein not found: ENSP00000443191
Protein not found: ENSP00000341867
10.534236267870579 % done
Protein not found: ENSP00000341867
21.068472535741158 % done
Protein not found: ENSP00000341867
31.60270880361174 % done
42.136945071482316 % done
Protein not found: ENSP00000361672
Protein not found: ENSP00000484912
52.6711813393529 % done
63.20541760722348 % done
73.73965387509406 % done
Protein not found: ENSP00000361672
84.27389014296463 % done
Protein not found: ENSP00000361672
94.80812641083521 % done


In [376]:
mix_ids = []
o_count = 0
prob_count = 0
cursor.execute('''SELECT id FROM gene_features''')
[mix_ids.append(x[0]) for x in cursor.fetchall()]
for i in mix_ids:
    cursor.execute('''SELECT COUNT (*) FROM paralogs WHERE id = ? AND vertebrate = "Yes"''', (i,))
    try:
        para_count = cursor.fetchone()[0]
    except IndexError:
        para_count = 0
    cursor.execute('''SELECT COUNT (*) FROM ohnologs_gene_ids WHERE id = ?''', (i,))
    try:
        o_count = cursor.fetchone()[0]
    except IndexError:
        o_count = 0
    if o_count > 4:
        prob_count += 1
        print(i, para_count, o_count)
    
print(prob_count)

ENSG00000143320 5 5
ENSG00000134183 5 5
ENSG00000173369 11 9
ENSG00000179546 5 5
ENSG00000132692 7 7
ENSG00000168243 8 5
ENSG00000188004 9 6
ENSG00000070886 13 13
ENSG00000126091 7 7
ENSG00000116329 10 10
ENSG00000187513 15 8
ENSG00000170128 8 8
ENSG00000135773 8 5
ENSG00000132702 7 7
ENSG00000133216 13 13
ENSG00000196189 12 5
ENSG00000162909 8 5
ENSG00000243284 9 6
ENSG00000185278 11 10
ENSG00000082482 5 5
ENSG00000159189 11 9
ENSG00000162444 5 5
ENSG00000170075 6 6
ENSG00000203697 8 5
ENSG00000117152 9 7
ENSG00000117069 5 5
ENSG00000174502 9 5
ENSG00000121634 15 8
ENSG00000184005 5 5
ENSG00000119535 6 5
ENSG00000050628 7 7
ENSG00000143537 14 5
ENSG00000065135 5 5
ENSG00000143248 9 7
ENSG00000172380 8 5
ENSG00000183317 13 13
ENSG00000143333 9 6
ENSG00000171488 6 5
ENSG00000171492 6 5
ENSG00000081985 6 5
ENSG00000265107 15 8
ENSG00000143127 10 7
ENSG00000171517 10 10
ENSG00000198835 15 8
ENSG00000135824 9 6
ENSG00000143167 9 7
ENSG00000197147 6 5
ENSG00000171812 11 9
ENSG00000181773 10

In [349]:
cursor.execute('''CREATE TABLE IF NOT EXISTS ohnologs_gene_ids(id TEXT, ohno_id TEXT, tax_group TEXT, vertebrate TEXT)''')
cursor.execute('''SELECT * FROM ohnologs''')
for y in cursor.fetchall():
    ID = y[2]
    ohno_p = y[1].split(' ')
    for o in ohno_p:
        cursor.execute('''SELECT id FROM gene_features_all WHERE prot_id = ?''', (o,))
        try:
            o_ID = cursor.fetchall()[0][0]
        except IndexError:
            print(o)
            next
        cursor.execute('''SELECT tax_group, vertebrate FROM paralogs WHERE id = ? AND para = ?''', (ID, o_ID))
        try:
            tax, vert = cursor.fetchall()[0]
            cursor.execute('''INSERT INTO ohnologs_gene_ids(id,ohno_id, tax_group, vertebrate) VALUES (?,?,?,?)''', (ID, o_ID, tax, vert))
        except IndexError:
            print(ID, o_ID)


ENSG00000134873 ENSG00000013297
ENSG00000068137 ENSG00000145555
ENSG00000188910 ENSG00000176402
ENSG00000188910 ENSG00000177291
ENSG00000092758 ENSG00000124749
ENSG00000182578 ENSG00000282278
ENSG00000183484 ENSG00000164220
ENSG00000183484 ENSG00000164251
ENSG00000183484 ENSG00000182162
ENSG00000183484 ENSG00000181104
ENSG00000183484 ENSG00000126266
ENSG00000138606 ENSG00000255872
ENSG00000147145 ENSG00000186912
ENSG00000147145 ENSG00000175591
ENSG00000147145 ENSG00000171631
ENSG00000066405 ENSG00000013297
ENSG00000163145 ENSG00000173372
ENSG00000163145 ENSG00000173918
ENSG00000006016 ENSG00000112964
ENSG00000006016 ENSG00000113494
ENSG00000198835 ENSG00000159248
ENSG00000198835 ENSG00000183153
ENSG00000138131 ENSG00000164099
ENSG00000138131 ENSG00000179954
ENSG00000138131 ENSG00000073754
ENSG00000138131 ENSG00000013725
ENSG00000138131 ENSG00000187908
ENSG00000138131 ENSG00000146700
ENSG00000242419 ENSG00000248383
ENSG00000113494 ENSG00000006016
ENSP00000443191
ENSG00000198642 ENSG0000

In [367]:
cursor.execute('''DELETE FROM ohnologs_gene_ids WHERE tax_group != "Vertebrata" AND tax_group != "Euteleostomi"''')

In [369]:
cursor.execute('''SELECT COUNT (DISTINCT id) FROM ohnologs_gene_ids''')
cursor.fetchall()

[(6049,)]

In [397]:
# cursor.execute('''CREATE TABLE IF NOT EXISTS ohnologs_t(id TEXT, ohno_id TEXT)''')
# with open('Downloads/takashi_ohnologs.csv', 'r') as file:
#     print(file.readline())
#     print(file.readline())
#     for line in file:
#         line = line.strip().split(',')
#         cursor.execute('''INSERT INTO t_ohnologs(id, ohno_id) VALUES(?,?)''', (line[0], line[3]))
#     db.commit()
cursor.execute('''UPDATE gene_features SET dup_type_t = "-"''')
cursor.execute('''UPDATE gene_features SET dup_type_t = "singleton" WHERE id NOT IN 
                (SELECT id FROM paralogs WHERE vertebrate = "Yes")''')
ids = []

cursor.execute('''SELECT id FROM gene_features WHERE dup_type_t != "singleton"''')
[ids.append(x[0]) for x in cursor.fetchall()]

count = 0
count_d = 0
count_check = 0
for i in ids:
    WGD = 'No'
    SSD = 'No'
    paras = []
    ohnos = []
    cursor.execute('SELECT para FROM paralogs WHERE id ==? AND vertebrate = "Yes"', (i,))
    [paras.append(x[0]) for x in cursor.fetchall()]
    cursor.execute('SELECT ohno_id FROM t_ohnologs WHERE id == ?', (i,))
    try:
        [ohnos.append(x[0]) for x in cursor.fetchall()]        
        for p in paras:
            if p in ohnos:
                WGD = 'Yes'
                WGD_count += 1
            elif p not in g_ohnos:
                SSD = 'Yes'
                SSD_count += 1
    except IndexError:
        WGD = 'No'
        SSD = 'Yes'
    if (WGD == 'Yes') and (SSD == 'Yes'):
        count += 1
        cursor.execute('''UPDATE gene_features SET dup_type_t = "WGD/SSD" WHERE id ==?''', (i,))
    elif WGD == 'Yes':
        cursor.execute('''UPDATE gene_features SET dup_type_t = "WGD" WHERE id == ?''', (i,))
    elif SSD == 'Yes':
        cursor.execute('''UPDATE gene_features SET dup_type_t = "SSD" WHERE id == ?''', (i,))
    else:
        print(i)
        print('Problem!')
    count_check +=1
    if count_check % 500 == 0:
        print(count_check * 100 /len(ids))

4.110489970404473
8.220979940808945
12.331469911213416
16.44195988161789
20.55244985202236
24.662939822426832
28.773429792831305
32.88391976323578
36.994409733640246
41.10489970404472
45.21538967444919
49.325879644853664
53.43636961525814
57.54685958566261
61.65734955606708
65.76783952647156
69.87832949687603
73.98881946728049
78.09930943768497
82.20979940808944
86.32028937849391
90.43077934889838
94.54126931930286
98.65175928970733


In [11]:
cursor.execute('''ALTER TABLE gene_features ADD COLUMN w_exp REAL''')

In [58]:
#get weighted "cost" in each tissue then take max weighed "cost"
import time
import numpy as np

ids, tissues = [], []
cursor.execute('''SELECT id FROM gene_features''')
[ids.append(i[0]) for i in cursor.fetchall()]
cursor.execute('''SELECT DISTINCT tissue FROM Transcript_expression_data''')
[tissues.append(x[0]) for x in cursor.fetchall()]

ti = time.time()
count = 0
val_list = []

for i in ids:
    count += 1
    cost_list = []
    cursor.execute('''SELECT Transcript_expression_data.trans_id, avg_exp, cds, tissue 
                        FROM Transcript_expression_data 
                        INNER JOIN gene_features_all 
                        ON Transcript_expression_data.trans_id = gene_features_all.trans_id 
                        WHERE Transcript_expression_data.id = ?''', (i,))
    results = cursor.fetchall()

    for t in tissues:
        try:
            values = [x[2] for x in results if x[3] == t]
            weights = [x[1] for x in results if x[3] == t]
            w_avg = np.average(values, weights=weights)
            cost_list.append(w_avg)
        except ZeroDivisionError:
            next
    try:
        val_list.append((max(cost_list), i))
    except ValueError:
        val_list.append((-1,i))
#     print('Appending values:', time.time()-t5) # 3 x10-5 seconds
    if (count % 500 == 0) or (count == len(ids)):
        cursor.executemany('''UPDATE gene_features SET w_exp = ? WHERE id =?''', val_list)
        print((count/len(ids))*100, '% done. It took', (float(time.time())-float(ti))/60, 'minutes')
        ti = time.time()
        count = 0
        val_list.clear()

2.4658480051289637 % done. It took 21.596643682320913 minutes
2.4658480051289637 % done. It took 21.482976015408834 minutes
2.4658480051289637 % done. It took 21.577919149398802 minutes
2.4658480051289637 % done. It took 21.661167716979982 minutes
2.4658480051289637 % done. It took 21.62441439628601 minutes
2.4658480051289637 % done. It took 21.5343355178833 minutes
2.4658480051289637 % done. It took 20.357250197728476 minutes
2.4658480051289637 % done. It took 20.30378384987513 minutes
2.4658480051289637 % done. It took 20.136840518315633 minutes
2.4658480051289637 % done. It took 19.85604245265325 minutes
2.4658480051289637 % done. It took 21.37819060087204 minutes
2.4658480051289637 % done. It took 21.50435978571574 minutes
2.4658480051289637 % done. It took 21.52489720185598 minutes
2.4658480051289637 % done. It took 21.77854701677958 minutes
2.4658480051289637 % done. It took 21.80896906455358 minutes
2.4658480051289637 % done. It took 20.95266273419062 minutes
2.4658480051289637 

In [25]:
cursor.execute('''CREATE TABLE IF NOT EXISTS macaque_paralogs(id TEXT, m_id TEXT, dn_ds REAL)''')
import requests, json, time
cursor.execute('''SELECT id FROM gene_features''')
ids = [x[0] for x in cursor.fetchall()]

timer = time.time()
count, counter = 0, 0
t = time.time()
values = []

for i in ids:
    orth_dict = {}
    if (time.time()-t <= 1) and (count >= 14):
        time.sleep(1)
        t = time.time()
        count = 0
        url = 'http://rest.ensembl.org/homology/id/' + i + '?type=orthologues;target_species=macaque;sequence=none;'
        headers = {'Content-type':'application/json'}
        res = requests.get(url, headers=headers).json()
        if res['data'][0]['homologies'] != []:
            orth_dict = res['data'][0]['homologies']
        else:
            next
        if len(orth_dict) == 1:
            for x in orth_dict:
                m_id = x['target']['id']
                dnds = x['dn_ds']
        values.append((i, m_id, dnds))
            
    else:
        url = 'http://rest.ensembl.org/homology/id/' + i + '?type=orthologues;target_species=macaque;sequence=none;'
        headers = {'Content-type':'application/json'}
        res = requests.get(url, headers=headers).json()
        if res['data'][0]['homologies'] != []:
            orth_dict = res['data'][0]['homologies']
        else:
            next
        if len(orth_dict) == 1:
            for x in orth_dict:
                m_id = x['target']['id']
                dnds = x['dn_ds']
        values.append((i, m_id, dnds))
    count += 1  
    counter += 1
    if (counter % 500 == 0) or (counter == len(ids)):
        print(counter, time.time()- timer)
        timer = time.time()
        cursor.executemany('''INSERT INTO macaque_paralogs(id,m_id,dn_ds) VALUES (?,?,?)''', values)
        values.clear()

500 37.769503116607666
1000 57.20268201828003
1500 65.02598810195923
2000 63.63481020927429
2500 65.22184300422668
3000 63.58103704452515
3500 59.83474516868591
4000 61.589075803756714
4500 72.98290491104126
5000 67.87666296958923
5500 66.74537801742554
6000 61.81927990913391
6500 64.3347098827362
7000 62.7044517993927
7500 60.90830206871033
8000 62.16087317466736
8500 64.2503809928894
9000 60.78619313240051
9500 58.615514039993286
10000 58.86919593811035
10500 57.222445011138916
11000 62.403748989105225
11500 62.72648596763611
12000 62.24689292907715
12500 61.30013704299927
13000 59.032795906066895
13500 60.488818883895874
14000 62.029731035232544
14500 63.190608978271484
15000 60.690735816955566
15500 63.51051592826843
16000 59.49983501434326
16500 60.104926109313965
17000 60.4296932220459
17500 58.42039394378662
18000 58.829643964767456
18500 63.579444885253906
19000 62.352333068847656
19500 59.66394114494324
20000 60.544914960861206
20277 31.88641905784607


In [7]:
print(one2one_count)

16575


In [71]:
db.commit()
cursor.execute('''SELECT cds_len, cds_exp, w_exp FROM gene_features WHERE cds_exp != -1 
                AND cds_exp != w_exp
                AND cds_len != w_exp''')
[print(x) for x in cursor.fetchall()]

(2433, 2433, 2426.783289624349)
(1158, 1158, 1140.4614560506452)
(3744, 3744, 3638.010478186321)
(1200, 1155, 1122.4665731340915)
(1122, 1122, 1056.340878005054)
(1776, 1776, 1767.2282330226856)
(1806, 1806, 1806.0000000000002)
(1503, 1503, 1485.156836461126)
(1386, 1386, 1399.346905194429)
(309, 213, 213.00000000000003)
(3720, 3720, 3719.5105763141228)
(2409, 2409, 2393.748496021735)
(3144, 3144, 3144.0000000000005)
(2856, 2583, 2705.888847369935)
(2289, 995, 1749.0416312418522)
(927, 927, 927.0000000000001)
(417, 417, 414.2324035467641)
(396, 396, 388.5151119157821)
(1200, 1200, 1178.6523257659946)
(3126, 468, 1851.7507763606493)
(1047, 1047, 956.1430148344884)
(741, 585, 637.7323943661971)
(2211, 1695, 1716.2983705581862)
(1842, 1557, 1646.9699999999998)
(1740, 1740, 1473.8044949905227)
(873, 873, 871.7390757811564)
(858, 849, 821.8265592151364)
(4215, 3936, 3935.938442211055)
(4470, 4437, 4156.616688905523)
(4938, 455, 1888.3541321458163)
(1641, 1401, 1517.2718446601941)
(3798, 379

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [69]:
cursor.execute('''SELECT * FROM gene_features WHERE cds_exp == 129''')
print(cursor.fetchall())

[('ENSG00000134376', 1, 197268204, 197478455, 210252, 4221, 129, 'CRB1', 2.531644643925793, -0.019, 0.5217391304347826, 'SSD', 'Yes', 46.0, 0.0006135494549397865, 'WGD', 1806.642857142857), ('ENSG00000269476', 19, 57876765, 57916591, 39827, 129, 129, 'AC010326.2', -1.0, None, None, 'singleton', '-', 0.9, 0.0032390087126823513, 'singleton', 129.0), ('ENSG00000272162', 6, 10747794, 10930423, 182630, 129, 129, 'AL024498.2', -1.0, None, None, 'singleton', '-', 88.0, 0.0007063461643760609, 'singleton', 129.0), ('ENSG00000106443', 7, 10973872, 11169630, 195759, 2847, 129, 'PHF14', 16.724504504504505, 0.169, 0.0, 'singleton', 'Yes', 93.0, 0.0006589735337838873, 'singleton', 1982.2745098039215), ('ENSG00000204165', 'X', 71103889, 71106788, 2900, 552, 129, 'CXorf65', 65.35053922779922, 0.121, 0.0, 'singleton', 'No', 2.0, 0.04448275862068966, 'singleton', 353.0292352669642)]


In [14]:
db.commit()
cursor.close()
db.close()

In [269]:
cursor.execute('''SELECT COUNT (DISTINCT id) FROM gene_features_all''')
cursor.fetchall()[0][0]

20277

In [ ]:
#gene expression (fix- setting as median)
count = 0
pcount = 0
import re
import numpy as np
import time
t1 = time.time()
cursor.execute('''BEGIN TRANSACTION''')
cursor.execute('CREATE TABLE IF NOT EXISTS Gene_expression_data(id TEXT, name TEXT, tissue TEXT, avg_exp REAL)')
cursor.execute('''COMMIT''')
with open('Downloads/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.txt') as expression_file:
    for i in range(2):
        expression_file.readline()
    sample_ids = expression_file.readline().strip().split('\t')[2:]
    #gets where each tissue starts and ends in the sample ids
    tissue_index = {}
    old_tissue = ''
    for i in range(0,len(sample_ids)):
        #get the tissue corresponding to each id, make a dictionary of which indexes in the line belong to each tissue
        cursor.execute('''BEGIN TRANSACTION''')
        cursor.execute('''SELECT tissue FROM sample_att WHERE GTex_id == ?''', (sample_ids[i],))
        tissue = cursor.fetchall()[0][0]
        cursor.execute('''COMMIT''')
        if tissue == old_tissue:
            tissue_index[tissue].append(i)
        else:
            tissue_index[tissue] = [i]
            old_tissue = tissue
    print(time.time()-t1)
    #each line is a gene
    update_count = 0
    val_list = []
    pcount = 0
    print('starting')
    t = time.time()
    for line in expression_file:
        count += 1
        av_dict = {}
        line = line.strip().split('\t')
        #gives the gene id without the version number
        gene = re.search(r'(.*)\.(.*)', line[0]).group(1)
        exp_values = line[2:]
        for i in range(0,len(exp_values)):
             exp_values[i] = float(exp_values[i])
        for tissue in tissue_index:
            mini = min(tissue_index[tissue])
            maxi = max(tissue_index[tissue]) + 1
            #get the average expression for each tissue
            av_dict[tissue] = np.median(exp_values[mini:maxi])
            cursor.execute('''BEGIN TRANSACTION''')
            cursor.execute('''UPDATE Gene_expression_data SET avg_exp = ? WHERE id == ? AND tissue == ?''', (av_dict[tissue], gene, tissue))
            cursor.execute('''COMMIT''')
        if count % 200 == 0:  
            print(str(count), time.time() - t)
            t = time.time()
        #need to add the expression data from other sources 
db.commit()

49.84485101699829
starting
200 3138.00630903244
400 3116.563446044922
600 3120.882938146591
800 3111.3546059131622
1000 3109.666086912155
1200 3110.403367996216
1400 3109.669158935547
1600 3110.517269849777
1800 3111.057408809662
2000 3110.7360548973083
2200 3110.046371936798
2400 3110.7923460006714
2600 3110.9394669532776
2800 3111.5114040374756
3000 3110.6562011241913
3200 3109.270812034607
3400 3110.3184311389923
3600 3109.6065690517426
3800 3110.827378988266
4000 3110.777886867523
4200 3110.2127141952515
4400 3113.10306596756
4600 3118.8885300159454
4800 3110.167349100113
5000 3110.654494047165
5200 3110.3967978954315
5400 3110.756507873535
5600 3110.8870599269867
5800 3108.883094072342
6000 3109.8299939632416
6200 3111.7050189971924
6400 3113.800707101822
6600 3110.1287751197815
6800 3111.2369141578674
7000 3109.739989042282
7200 3111.5417630672455
7400 3107.232134103775
7600 3110.9773919582367
7800 3111.856598854065
8000 3110.910651922226
8200 3109.568344116211
8400 3111.18384504

KeyboardInterrupt: 

In [ ]:
#transcript expression (fix)
count = 0
cursor.execute('CREATE TABLE IF NOT EXISTS Transcript_expression_data(id TEXT, trans_id TEXT, tissue TEXT, avg_exp REAL)')

with open('Downloads/GTEx_Analysis_2016-01-15_v7_RSEMv1.2.22_transcript_tpm.txt') as t_exp_file:
    #the first two items aren't ids
    sample_ids_2 = t_exp_file.readline().strip().split('\t')[2:]
    tissue_index = {}
    old_tissue = ''
    for i in range(0,len(sample_ids_2)):
        cursor.execute('''SELECT tissue FROM sample_att WHERE GTex_id == ?''', (sample_ids[i],))
        tissue = cursor.fetchall()[0][0]
        if tissue == old_tissue:
            tissue_index[tissue].append(i)
        else:
            tissue_index[tissue] = [i]
            old_tissue = tissue
    for line in t_exp_file:
        t = time.time()
        av_dict = {}
        count += 1
        line = line.strip().split('\t')        
        transcript = re.search(r'(.*)\.(.*)', line[0]).group(1)
        gene = re.search(r'(.*)\.(.*)', line[1]).group(1)
        t_exp_values = line[2:]
        for i in range(0,len(t_exp_values)):
             t_exp_values[i] = float(t_exp_values[i])
        for tissue in tissue_index:
            mini = min(tissue_index[tissue])
            maxi = max(tissue_index[tissue]) + 1 #because of 'up to but not including' in list slicing
            #get the average expression for each tissue
            av_dict[tissue] = np.median(t_exp_values[mini:maxi])
            val_list.append((av_dict[tissue], transcript, tissue))
        if count % 500 == 0:
            cursor.executemany('''UPDATE Transcript_expression_data SET avg_exp = ? WHERE trans_id = ? AND tissue = ?''', val_list)
            print(str(count), time.time()-t)
            t = time.time()
            val_list.clear()
    cursor.executemany('''UPDATE Transcript_expression_data SET avg_exp = ? WHERE trans_id = ? AND tissue = ?''', val_list)

db.commit()

In [28]:
cursor.execute('''ALTER TABLE gene_features ADD COLUMN unique_domains INTEGER''')

In [202]:
# unique protein domains
import requests, json, time

cursor.execute('''SELECT DISTINCT id FROM gene_features_all''')
gene_list = [x[0] for x in cursor.fetchall()]
actual_count = 0
t1 = time.time()
t = time.time()
count = 0
for g in gene_list:
    actual_count += 1
    dom_list = []
    cursor.execute('''SELECT prot_id FROM gene_features_all WHERE id =?''', (g,))
    prot_list = [x[0] for x in cursor.fetchall()]
    for p in prot_list:
        u_time = time.time()
        count += 1
        if time.time() - t <= 1 and count >= 10:
            time.sleep(1)
            t = time.time()
            count = 0
            url = "https://rest.ensembl.org/overlap/translation/" + p + "?type=Pfam"
            headers = { "Content-Type" : "application/json"}
            r = requests.get(url, headers=headers).json()
            if r != []:
                [dom_list.append(x['id']) for x in r]
            else:
                next
        else:
            url = "https://rest.ensembl.org/overlap/translation/" + p + "?type=Pfam"
            headers = { "Content-Type" : "application/json"}
            r = requests.get(url, headers=headers).json()
            if r != []:
                [dom_list.append(x['id']) for x in r]
            else:
                next
    u = len(set(dom_list))
    cursor.execute('''UPDATE gene_features SET unique_domains = ? WHERE id = ?''', (u, g))
    if actual_count % 100 == 0:
        print(actual_count, ':', time.time()-t1, 'seconds')

100 : 137.15532994270325 seconds
200 : 246.23370099067688 seconds
300 : 400.0122470855713 seconds
400 : 562.5888018608093 seconds
500 : 666.8423728942871 seconds
600 : 776.8492419719696 seconds
700 : 936.773843050003 seconds
800 : 1062.715646982193 seconds
900 : 1176.6543579101562 seconds
1000 : 1312.6162889003754 seconds
1100 : 1451.702630996704 seconds
1200 : 1625.3460459709167 seconds
1300 : 1740.23268699646 seconds
1400 : 1879.0334739685059 seconds
1500 : 2036.0867879390717 seconds
1600 : 2172.081733942032 seconds
1700 : 2297.875331878662 seconds
1800 : 2421.4697358608246 seconds
1900 : 2532.3048799037933 seconds
2000 : 2633.0978479385376 seconds
2100 : 2753.196188926697 seconds
2200 : 2894.0006399154663 seconds
2300 : 3033.930130958557 seconds
2400 : 3216.0110080242157 seconds
2500 : 3352.857759952545 seconds
2600 : 3474.9454460144043 seconds
2700 : 3614.0387239456177 seconds
2800 : 3775.7055439949036 seconds
2900 : 3941.5196928977966 seconds
3000 : 4084.4113149642944 seconds
3100

In [194]:
cursor.execute('''ALTER TABLE gene_features ADD COLUMN domains INTEGER''')

In [200]:
#absolute number of protein domains (mapped from protein coordinates to genomic coordinates)
import itertools
cursor.execute('''SELECT DISTINCT id FROM gene_features_all''')
gene_list = [x[0] for x in cursor.fetchall()]
actual_count = 0
t = time.time()
timer = time.time()
count = 0
counter = 0
for g in gene_list:
    counter += 1
    domain_list = []
    list_count = -1
    
    cursor.execute('''SELECT prot_id FROM gene_features_all WHERE id =?''', (g,))
    prot_list = [x[0] for x in cursor.fetchall()]
    for p in prot_list:
        count += 1

        if time.time() - t <= 1 and count >= 15:
            
            time.sleep(1)
            t = time.time()
            count = 0
            
            url = "https://rest.ensembl.org/overlap/translation/" + p + "?type=Pfam"
            headers = { "Content-Type" : "application/json"}
            r = requests.get(url, headers=headers).json()
            
            if r != []:
                #go through all domains in the protein
                for y in r:
                    count += 1
                    list_count += 1
                    s, e, d = y['start'], y['end'], y['id']
                    url2 = "https://rest.ensembl.org/map/translation/" + p + "/" + str(s) + ".." + str(e) + "?"
                    r2 = requests.get(url2, headers=headers).json()
                    #go through all the genomic coordinates the domain maps to
                    domain_list.append([d])
                    for z in r2['mappings']:
                        start = min(z['start'], z['end'])
                        end = max(z['start'], z['end'])
                        domain_list[list_count].append((start, end))
                        
                        
            else:
                next
                
        elif time.time()-t > 1:
        
            t = time.time()
            count = 0
            
            url = "https://rest.ensembl.org/overlap/translation/" + p + "?type=Pfam"
            headers = { "Content-Type" : "application/json"}
            r = requests.get(url, headers=headers).json()
            
            if r != []:
                for y in r:
                    count += 1
                    list_count += 1
                    s, e, d = y['start'], y['end'], y['id']
                    url2 = "https://rest.ensembl.org/map/translation/" + p + "/" + str(s) + ".." + str(e) + "?"
                    r2 = requests.get(url2, headers=headers).json()
                    #go through all the genomic coordinates the domain maps to
                    domain_list.append([d])
                    for z in r2['mappings']:
                        start = min(z['start'], z['end'])
                        end = max(z['start'], z['end'])
                        domain_list[list_count].append((start, end))
            else:
                next
        else:
            
            url = "https://rest.ensembl.org/overlap/translation/" + p + "?type=Pfam"
            headers = { "Content-Type" : "application/json"}
            r = requests.get(url, headers=headers).json()
            if r != []:
                for y in r:
                    list_count += 1
                    count += 1
                    s, e, d = y['start'], y['end'], y['id']
                    url2 = "https://rest.ensembl.org/map/translation/" + p + "/" + str(s) + ".." + str(e) + "?"
                    r2 = requests.get(url2, headers=headers).json()
                    #go through all the genomic coordinates the domain maps to
                    domain_list.append([d])
                    for z in r2['mappings']:
                        start = min(z['start'], z['end'])
                        end = max(z['start'], z['end'])
                        domain_list[list_count].append((start, end))
            else:
                next
                
    no_domains = merge_domains(domain_list)
    cursor.execute('''UPDATE gene_features SET domains = ? WHERE id == ?''', (no_domains, g))
    
    if counter % 100 == 0:
        print(counter, "done:", time.time()-timer, "seconds")


100 done: 225.98121190071106 seconds
200 done: 404.2602620124817 seconds
300 done: 693.9938340187073 seconds
400 done: 964.207841873169 seconds
500 done: 1136.6027438640594 seconds
600 done: 1363.0471329689026 seconds
700 done: 1734.79438996315 seconds
800 done: 2001.1276528835297 seconds
900 done: 2193.161670923233 seconds
1000 done: 2431.2231678962708 seconds
1100 done: 2673.64027094841 seconds
1200 done: 2991.899379968643 seconds
1300 done: 3204.20397400856 seconds
1400 done: 3443.13813495636 seconds
1500 done: 3740.295740842819 seconds
1600 done: 3980.11541891098 seconds
1700 done: 4192.855660915375 seconds
1800 done: 4424.930265903473 seconds
1900 done: 4617.2063710689545 seconds
2000 done: 4784.700680017471 seconds
2100 done: 5058.752866983414 seconds
2200 done: 5355.078042984009 seconds
2300 done: 5623.225549936295 seconds
2400 done: 5913.200388908386 seconds
2500 done: 6150.101665973663 seconds
2600 done: 6365.641799926758 seconds
2700 done: 6605.356189966202 seconds
2800 done:

In [203]:
db.commit()

In [ ]:
#structure of domain_list
test = [['a', (1,10), (12,20)], ['a', (43,50), (51,70)], ['a', (1,11), (12,21)], ['a', (1,10), (12,19)], 
        ['b', (1,20), (21,40)], 
        ['c', (30, 35), (41, 42), (51,56)], ['c', (70,100), (110,130), (150,160)]]

In [196]:
def merge_domains(map_list):
    import itertools
    ret_list = []
    same_domain = {}
    #remove completely identical values beforehand
    map_list = [list(x) for x in set([tuple(x) for x in map_list])] #lists aren't hashable, convert to tuple first
    domain_number = 0
    for ID in set([x[0] for x in map_list]):
        same_domain = {}
        #get coordinate list for each unique ID
        cList = [y[1:] for y in map_list if y[0] == ID]
        #if there's more than one set then just set domains = 1
        if len(cList) > 1:
            for pair in itertools.combinations(cList, 2): #gives all possible pairings of domains across all proteins
                overlap = 'No'
                d1 = pair[0]
                d2 = pair[1]
                #compare every coordinate pair for each domain with every other coordinate pair
                for loc in d1:
                    s = loc[0]
                    e = loc[1]
                    for loc2 in d2:
                        s2 = loc2[0]
                        e2 = loc2[1]
                        #check for overlap
                        if (range(s,e) in range(s2,e2)) or (range(s2,e2) in range(s,e)) or (range(s,e) == range(s2,e2)) or (s < s2 < e) or (s2 < s < e2) or (s < e2 < e) or (s2 < e < e2):
                            overlap = 'Yes'
                #set the dict value for this domain pair to 'Same' if they overlap and 'Different' if they don't
                if overlap == 'Yes':
                    same_domain[(cList.index(d1), cList.index(d2))] = 'Same'
                else:
                    same_domain[(cList.index(d1), cList.index(d2))] = 'Different'
            #list of all the values in the tuple keys
            u_d_list = [x for x in range(0,len(cList))]
#             for each domain value look for pairs it's in in the keys, 
#             then set the other value in the pair = to its partner in u_d_list if value for the key is 'Same'
            for i in u_d_list:
                for key in same_domain:
                    if key[0] == i and same_domain[key] == 'Same':
                        u_d_list = [i if u == key[1] else u for u in u_d_list]
                    elif key[1] == i and same_domain[key] == 'Same':
                        u_d_list = [i if u == key[0] else u for u in u_d_list]
            #number of unique values in u_d_list is the number of domains- values describing the same domain have replaced each other
            domains = len(set(u_d_list))
        else:
            domains = 1
        #increase the total number of domains by the amount with this ID
        domain_number = domain_number + domains
    return(domain_number)            


In [3]:
#cursor.execute('''ALTER TABLE gene_features ADD COLUMN gc_content REAL''')
with open('gc_content.txt', 'r') as gc_file:
    for line in gc_file:
        line = line.strip().split('\t')
        gene, gc = line
        cursor.execute('''UPDATE gene_features SET gc_content = ? WHERE id == ?''', (gc, gene))
db.commit()

In [31]:
#get CDS sequence and GC content of the third codon
import requests, time
cursor.execute('''ALTER TABLE gene_features ADD COLUMN gc3 REAL''')
cursor.execute('SELECT id FROM gene_features')
ids = [x[0] for x in cursor.fetchall()]
h={ "Content-Type" : "text/plain"}
count = 0
t = time.time()
for i in ids:
    count += 1
    url = "http://rest.ensembl.org/sequence/id/" + i + "?type=cds;multiple_sequences=1"
    r = requests.get(url, headers=h).text
    #get longest CDS, multiple sequences are returned separated by newline
    cds = max(r.split('\n'), key=len)
    #start at 3rd position and go in steps of 3
    third_codon = cds[2::3]
    g_count = third_codon.count('G')
    c_count = third_codon.count('C')
    gc_count = g_count + c_count
    gc_percent = (gc_count/len(third_codon))
    cursor.execute('''UPDATE gene_features SET gc3 = ? WHERE id == ?''', (gc_percent, i))
    if count % 100 == 0:
        print(count, time.time()-t)

100 15.626970052719116
200 29.30663800239563
300 45.04137992858887
400 60.3992440700531
500 73.44809699058533
600 87.75618195533752
700 103.28217697143555
800 117.4013569355011
900 131.99093294143677
1000 146.53352689743042
1100 161.4260959625244
1200 177.27885103225708
1300 191.7215220928192
1400 206.51942896842957
1500 222.1285479068756
1600 236.5111939907074
1700 251.39655995368958
1800 266.432501077652
1900 280.4366900920868
2000 294.86882495880127
2100 308.72747898101807
2200 324.77560901641846
2300 345.31098890304565
2400 363.89105701446533
2500 379.66497111320496
2600 396.1403090953827
2700 410.1856861114502
2800 427.8821749687195
2900 444.5512490272522
3000 459.03479290008545
3100 474.573117017746
3200 489.6069059371948
3300 511.0329658985138
3400 527.1689350605011
3500 542.1856799125671
3600 556.9508709907532
3700 572.4071490764618
3800 586.2675380706787
3900 601.0440258979797
4000 616.4025120735168
4100 632.8572020530701
4200 648.2146980762482
4300 663.610044002533
4400 679.1

In [7]:
#get regulatory motifs within a gene and 5kb each side
cursor.execute('''ALTER TABLE gene_features ADD COLUMN motif_number_1k INTEGER''')
import requests, json, time
cursor.execute('''SELECT start,end FROM gene_features''')
gene_locs = cursor.fetchall()
count = 0
counter = 0
count_t = time.time()
t = time.time()

for i in ids:
    count += 1
    counter += 1
    if time.time()-t <= 1 and count >= 15:
        
        time.sleep(0.5)
        t = time.time()
        count = 0
        
        cursor.execute('''SELECT chrom, start, end FROM gene_features WHERE id ==?''', (i,))
        l = cursor.fetcahll([0])
        test_loc1 = l[1]
        loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
        
        url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
        headers = { "Content-Type" : "application/json"}
        r = requests.get(url, headers=headers).json()
        
        if r != []:
            cursor.execute('''UPDATE gene_features SET motif_number_1k =? WHERE id == ?''', (len(r), i))
        else:
            cursor.execute('''UPDATE gene_features SET motif_number_1k = 0 WHERE id == ?''', (i,))
    elif time.time()-t > 1:
        
        t = time.time()
        count = 0
        
        cursor.execute('''SELECT chrom, start, end FROM gene_features WHERE id ==?''', (i,))
        l = cursor.fetchall()[0]
        loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
        
        url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
        headers = { "Content-Type" : "application/json"}
        r = requests.get(url, headers=headers).json()
        
        if r != []:
            cursor.execute('''UPDATE gene_features SET motif_number_1k =? WHERE id == ?''', (len(r), i))
        else:
            cursor.execute('''UPDATE gene_features SET motif_number_1k = 0 WHERE id == ?''', (i,))
    else:
        
        cursor.execute('''SELECT chrom, start, end FROM gene_features WHERE id ==?''', (i,))
        l = cursor.fetchall()[0]
        loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
        
        url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
        headers = { "Content-Type" : "application/json"}
        r = requests.get(url, headers=headers).json()
        
        if r != []:
            cursor.execute('''UPDATE gene_features SET motif_number_1k =? WHERE id == ?''', (len(r), i))
        else:
            cursor.execute('''UPDATE gene_features SET motif_number_1k = 0 WHERE id == ?''', (i,))
    if counter% 200 == 0:
        print(counter, time.time()-count_t, 'seconds since started')

200 28.010648012161255 seconds since started
400 89.42845702171326 seconds since started
600 116.26174211502075 seconds since started
800 140.57208013534546 seconds since started
1000 164.35743021965027 seconds since started
1200 187.1804871559143 seconds since started
1400 210.7606041431427 seconds since started
1600 234.3334801197052 seconds since started
1800 256.11987018585205 seconds since started
2000 278.8302540779114 seconds since started
2200 302.30669713020325 seconds since started
2400 324.29499101638794 seconds since started
2600 347.12424516677856 seconds since started
2800 369.93838000297546 seconds since started
3000 392.0380640029907 seconds since started
3200 413.6419150829315 seconds since started
3400 436.07618713378906 seconds since started
3600 457.31153106689453 seconds since started
3800 478.27139019966125 seconds since started
4000 500.571093082428 seconds since started
4200 522.7625930309296 seconds since started
4400 544.4261062145233 seconds since started
460

In [45]:
db.commit()

In [229]:
cursor.execute('''UPDATE gene_features SET trans_count = -1, intron_count = -1, intron_length = -1''')

In [4]:
# introns
    # total length: adding
    # number: get gene start end, define regions not covered by exons(exon start, end) in the gene
    # y/n: no gaps 
# transcript isoforms
    # number: number of unique sets of exon ids for all transcripts from a given gene
        #transcripts all have unique exon sets
    # number alt spliced: number that have different exons relative to the longest? (but need to check the ends)
# cursor.execute('''ALTER TABLE gene_features ADD COLUMN trans_count INTEGER''')
# cursor.execute('''ALTER TABLE gene_features ADD COLUMN intron_count INTEGER''')
# cursor.execute('''ALTER TABLE gene_features ADD COLUMN intron_length INTEGER''')

import requests, json, time
cursor.execute('''SELECT id FROM gene_features''')
ids = [x[0] for x in cursor.fetchall()]
h = {'Content-Type':'application/json'}
count = 0
timer = time.time()
for g in ids:
    
    introns = []
    exon_list = []
    u_exon_list = []
    cursor.execute('''SELECT start,end FROM gene_features WHERE id == ?''', (g,))
    start, end = cursor.fetchall()[0]
    cursor.execute('''SELECT trans_id FROM gene_features_all WHERE id == ?''', (g,))
    transcript_list = [x[0] for x in cursor.fetchall()]
    num_isoforms = len(transcript_list)

    for t in transcript_list:
        url = "http://rest.ensembl.org/overlap/id/" + t + "?feature=exon"
        r = requests.get(url, headers=h).json()
        if len(transcript_list) <= 15:
            time.sleep(0.1)
        for i in range(0, len(r)):
            if r[i]['Parent'] == t:
                exon_list.append((r[i]['start'], r[i]['end']))
    exon_list = sorted(exon_list, key=lambda x: x[0])
    for i in range(0, len(exon_list)):
        if i == 0:
            old_start = exon_list[i][0]
            old_end = exon_list[i][1]
        elif i == len(exon_list)-1 and exon_list[i][1] <= old_end:#add current interval to list if this is the last value
            u_exon_list.append((old_start, old_end))
        elif i == len(exon_list)-1 and exon_list[i][1] > old_end: #if the last entry on the list has a higher end than current interval
            u_exon_list.append((old_start, exon_list[i][1]))
        else:
            if exon_list[i][0] < old_end:
                old_end = exon_list[i][1]
            else:
                u_exon_list.append((old_start, old_end))
                old_start = exon_list[i][0]
                old_end = exon_list[i][1]
    for i in range(0, len(u_exon_list)):
        if i == 0:
            s = start
            if s != u_exon_list[i][0]: #case where there's an intron between gene start and 1st exon start
                introns.append((s,u_exon_list[i][0]))
                next_s = u_exon_list[i][1]
            else:
                next_s = u_exon_list[i][1] #if 1st exon start is gene start, set start of next intron and go to next
        elif i == len(u_exon_list)-1:
            introns.append((next_s, u_exon_list[i][0])) #for last exon, append preceding intron as normal then
            if end != u_exon_list[i][1]: #check for intron between end of last exon and gene end
                introns.append((u_exon_list[i][1], end))
        else:
            introns.append((next_s, u_exon_list[i][0]))
            next_s = u_exon_list[i][1]
    num_introns = len(introns)
    total_len_introns = sum([x[1]-x[0] for x in introns])
    cursor.execute('''UPDATE gene_features SET trans_count = ?, intron_count = ?, intron_length = ? WHERE id ==?''', (num_isoforms, num_introns, total_len_introns, g))
    count += 1
    if count% 500 == 0:
        print(count, time.time()-timer)
    

500 573.8495020866394
1000 1126.0583729743958
1500 1636.6510019302368
2000 2072.8139979839325
2500 2591.23890709877
3000 3238.3468520641327
3500 4076.5636761188507
4000 5254.920144081116
4500 5996.089576005936
5000 6652.38100194931
5500 7146.554493904114
6000 7712.279376029968
6500 8416.605736017227
7000 9098.90188908577
7500 9791.522596120834
8000 10496.779619932175
8500 11152.891531944275
9000 11972.097744941711
9500 12819.636781930923
10000 13503.584518909454
10500 14237.234493970871
11000 14907.314508914948
11500 15624.312053918839
12000 16197.677072048187
12500 16677.058881044388
13000 17166.471180915833
13500 17757.54750609398
14000 18456.93708705902
14500 19409.72564101219
15000 20076.534249067307
15500 20756.189068078995
16000 21425.822602033615
16500 22096.706048965454
17000 22713.41304898262
17500 23257.412008047104
18000 23769.463586091995
18500 24395.101490974426
19000 24896.461719036102
19500 25339.761785030365
20000 25840.68521809578


In [3]:
db.commit()

In [6]:
# cursor.execute('ALTER TABLE gene_features ADD COLUMN avg_intron REAL')
import time
count = 0
t = time.time()
cursor.execute('SELECT id, intron_count, intron_length FROM gene_features')
for g, c, l in cursor.fetchall():
    count += 1
    try:
        cursor.execute('''UPDATE gene_features SET avg_intron = ? WHERE id == ?''', (l/c, g))
    except ZeroDivisionError:
        cursor.execute('''UPDATE gene_features SET avg_intron = 0 WHERE id == ?''', (g,))
    if count % 500 == 0:
        print(count, time.time()-t)

500 8.821729898452759
1000 17.329663038253784
1500 25.520545959472656
2000 33.62408995628357
2500 41.75561308860779
3000 49.747997999191284
3500 58.05300688743591
4000 67.0113160610199
4500 75.28521394729614
5000 83.60820698738098
5500 91.8260428905487
6000 100.88104701042175
6500 109.7592658996582
7000 119.65880298614502
7500 128.82322907447815
8000 137.61012506484985
8500 146.1283700466156
9000 154.26188206672668
9500 163.01805901527405
10000 171.35189199447632
10500 179.8070089817047
11000 187.99212789535522
11500 195.94230103492737
12000 204.1178948879242
12500 212.0969979763031
13000 220.35721397399902
13500 229.35757088661194
14000 238.3377709388733
14500 247.95867204666138
15000 256.2412700653076
15500 264.52373909950256
16000 272.9529790878296
16500 281.4117488861084
17000 289.8549859523773
17500 298.3124330043793
18000 306.4464190006256
18500 314.98736691474915
19000 323.1154930591583
19500 331.91212701797485
20000 340.587769985199


In [7]:
db.commit()

In [3]:
cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type == "WGD"''')
print(cursor.fetchone())
cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type == "SSD"''')
print(cursor.fetchone())
cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type == "singleton"''')
print(cursor.fetchone())
cursor.execute('''SELECT COUNT (*) FROM gene_features WHERE dup_type == "WGD/SSD"''')
print(cursor.fetchone())

(4940,)
(6115,)
(8113,)
(1109,)


In [8]:
def tau(expression_list):
    expression_list = [float(x) for x in expression_list]
    m = max(expression_list)
    try:
        expression_list = [x/m for x in expression_list]
    except ZeroDivisionError:
        return(None)
    num_list = []
    for i in expression_list:
        num_list.append(1-i)
    s = sum(num_list)
    try:
        res = s/ (len(expression_list)-1)
    except ZeroDivisionError:
        print(expression_list)
        return(None)
    return(res)

In [3]:
import re
cursor.execute("SELECT id FROM gene_features")
ids = [x[0] for x in cursor.fetchall()]
with open('Downloads/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_median_tpm.gct', 'r') as med_file, \
    open('Downloads/fetal_exp.tsv', 'r') as f_file, open('Downloads/brain_dev_exp.tsv', 'r') as brain_file:
    exp_dict = {}
    for i in range(1,3):
        med_file.readline()
    header = med_file.readline().strip().split('\t')
    for line in med_file:
        line = line.strip().split('\t')
        gene = re.search(r'(.*)\.(.*)', line[0]).group(1)
        if gene in ids:
            exp_dict[gene] = line[2:]
    for i in range(1,5):
        f_file.readline()
    header = f_file.readline().strip().split('\t')
    f_list = []
    for thing in header:
        if 'fetal' in thing:
            f_list.append(header.index(thing))
    for line in f_file:
        line = line.strip('\n').split('\t')
        if line[0] in ids:
            start = line[0:2]
            start.extend([0 if x == '' else float(x) for x in line[2:]])
            line = start
            for i in f_list:
                try:
                    exp_dict[line[0]].append(line[i])
                except (KeyError, IndexError):
                    exp_dict[line[0]] = [line[i]]
    for i in range(1,6):
        brain_file.readline()
    for line in brain_file:
        line = line.strip('\n').split('\t')
        if line[0] in ids:
            start = line[0:2]
            start.extend([0 if x == '' else float(x) for x in line[2:]])
            line = start
            for i in range(2, len(line)):
                try:
                    exp_dict[line[0]].append(line[i])
                except KeyError:
                    exp_dict[line[0]] = [line[i]]

In [24]:
#cursor.execute('''ALTER TABLE gene_features ADD COLUMN specificity REAL''')
import time
cursor.execute('''UPDATE gene_features SET specificity = ?''', (None,))
count = 0
timer = time.time()
for k in exp_dict:
    expressed = 0
    for x in exp_dict[k]:
        if float(x) >= 1:
            expressed = 1
    if expressed == 1 and len(exp_dict[k]) == 238:
        t = tau(exp_dict[k])
        cursor.execute('''UPDATE gene_features SET specificity = ? WHERE id == ?''', (t, k))
    count += 1
    if count % 500 == 0:
        print(count,time.time()-timer)
db.commit()

500 5.14923095703125
1000 10.081080913543701
1500 15.186805963516235
2000 20.217002868652344
2500 25.758620977401733
3000 30.794413805007935
3500 35.64858388900757
4000 40.53430390357971
4500 45.45335292816162
5000 50.5180549621582
5500 56.058100938797
6000 62.521535873413086
6500 67.82228493690491
7000 73.10401487350464
7500 78.26165699958801
8000 83.2241427898407
8500 88.71039080619812
9000 93.88112688064575
9500 99.44430184364319
10000 105.07295083999634
10500 110.37523579597473
11000 115.65469789505005
11500 120.82026100158691
12000 126.30479383468628
12500 131.26413798332214
13000 136.32414293289185
13500 141.26088285446167
14000 146.94767594337463
14500 152.21639394760132
15000 157.57650899887085
15500 162.48983502388
16000 167.5906069278717
16500 172.92528080940247
17000 179.18400692939758
17500 184.33901000022888
18000 189.66084599494934
18500 195.0837938785553
19000 200.44081902503967
19500 206.04109597206116
20000 211.31324887275696


In [151]:
count = 0
for i in exp_dict:
    if len(exp_dict[i]) == 238:
        count += 1
print(count)

16404


In [74]:
cursor.execute('''SELECT id FROM gene_features WHERE specificity <= 0.35''')

In [75]:
low_spec = [x[0] for x in cursor.fetchall()]

In [6]:
import time
count = 0
#cursor.execute('''ALTER TABLE gene_features ADD COLUMN max_exp REAL''')
cursor.execute('''SELECT id FROM gene_features''')
ids = [x[0] for x in cursor.fetchall()]
timer = time.time()
for i in ids:
    try:
        expression_list = exp_dict[i]
        expression_list = [float(x) for x in expression_list]
        m = max(expression_list)
        cursor.execute('''UPDATE gene_features SET max_exp = ? WHERE id == ?''', (m, i))
        count += 1
        if count % 500 == 0:
            print(count, time.time()-timer)
    except KeyError:
        count += 1
        if count % 500 == 0:
            print(count, time.time()-timer)
        next

500 8.961426019668579
1000 17.127377033233643
1500 25.688976049423218
2000 34.742539167404175
2500 44.05445313453674
3000 53.08276700973511
3500 61.1956901550293
4000 70.22061800956726
4500 82.53903102874756
5000 91.6288230419159
5500 100.38769102096558
6000 108.98171496391296
6500 117.94463109970093
7000 126.82429504394531
7500 135.6059091091156
8000 144.23132300376892
8500 152.98640203475952
9000 161.949294090271
9500 170.36898517608643
10000 179.1329641342163
10500 188.18995308876038
11000 196.76868414878845
11500 205.31750011444092
12000 214.96977710723877
12500 224.17131400108337
13000 233.65360403060913
13500 242.92853212356567
14000 251.8859350681305
14500 260.8199291229248
15000 269.8536219596863
15500 278.9268729686737
16000 287.39314699172974
16500 298.1318609714508
17000 308.1335201263428
17500 316.9980511665344
18000 325.325795173645
18500 333.8383331298828
19000 342.44610500335693
19500 351.39410305023193
20000 361.02262806892395


In [28]:
cursor.execute('CREATE TABLE IF NOT EXISTS GO_terms(id TEXT, go TEXT, name TEXT, domains TEXT)')
# cursor.execute('ALTER TABLE gene_features ADD COLUMN go_id TEXT')
import time
c = 0
t = time.time()
with open('Downloads/GO_terms_by_gene.txt', 'r') as file:
    for line in file:
        line = line.strip().split('\t')
        try:
            gene, acc, name, domain = line
            cursor.execute('''INSERT INTO GO_terms(id,go,name,domains) VALUES (?,?,?,?)''', (gene,acc, name, domain))
            c += 1
            if c%1000 == 0:
                print(c, time.time()-t)
        except ValueError:
            c += 1
            if c%1000 == 0:
                print(c, time.time()-t)
            next

1000 3.3052520751953125
2000 6.55370306968689
3000 9.939260959625244
4000 13.192221879959106
5000 17.470398902893066
6000 21.032628059387207
7000 23.733402013778687
8000 26.067462921142578
9000 28.82955002784729
10000 31.893805027008057
11000 34.68308186531067
12000 37.73235297203064
13000 40.70966196060181
14000 43.640666007995605
15000 46.53732395172119
16000 49.46708106994629
17000 53.07179594039917
18000 55.79128694534302
19000 58.59182286262512
20000 61.329323053359985
21000 64.05121803283691
22000 67.06913304328918
23000 69.98810386657715
24000 73.15542888641357
25000 76.29733896255493
26000 79.31048393249512
27000 82.73120784759521
28000 85.27394604682922
29000 87.4583990573883
30000 90.55751705169678
31000 93.73108291625977
32000 96.8312258720398
33000 99.9385449886322
34000 102.8837058544159
35000 105.7010269165039
36000 108.69545388221741
37000 112.6702868938446
38000 115.57896900177002
39000 118.51723885536194
40000 122.06754088401794
41000 125.86718106269836
42000 129.08245

331000 1063.8965439796448
332000 1066.7305319309235
333000 1070.3043229579926
334000 1073.1409089565277
335000 1075.9352960586548
336000 1078.810392856598
337000 1081.8456189632416
338000 1084.1404249668121
339000 1086.8411819934845


In [ ]:
# nice example of filtering API results
# list comp sets values not from searched gene to 0
# filter out zero values from the list
r = list(filter(lambda x: x != 0, [x if x['Parent'] == i else 0 for x in r]))

In [12]:
cursor.execute('''SELECT id FROM gene_features WHERE dup_type == "WGD"''')
w_ids = [x[0] for x in cursor.fetchall()]
w_gos = []
for i in w_ids:
    cursor.execute('''SELECT go FROM GO_terms WHERE id == ?''', (i,))
    go_list = [x[0] for x in cursor.fetchall()]
    w_gos.extend(go_list)
term_dict = {}
for item in set(w_gos):
    w_term_dict[item] = w_gos.count(item)

NameError: name 'w_term_dict' is not defined

In [14]:
cursor.execute('''SELECT id FROM gene_features WHERE dup_type == "SSD"''')
s_ids = [x[0] for x in cursor.fetchall()]
s_gos = []
for i in s_ids:
    cursor.execute('''SELECT go FROM GO_terms WHERE id == ?''', (i,))
    go_list = [x[0] for x in cursor.fetchall()]
    s_gos.extend(go_list)
s_term_dict = {}
for item in set(s_gos):
    s_term_dict[item] = s_gos.count(item)

In [13]:
w_term_dict = {}
for item in set(w_gos):
    w_term_dict[item] = w_gos.count(item)

In [16]:
# [print(k, s_term_dict[k]) for k in s_term_dict]
s_term_dict['GO:0006810']

407

In [ ]:
import requests, json
cursor.execute('''SELECT id FROM gene_features''')
ids = [x[0] for x in cursor.fetchall()]
headers = {'Content-Type':'application/json'}
for i in ids:
    cursor.execute('''SELECT trans_id FROM gene_features_all WHERE id == ?''', (i,))
    t_ids = [x[0] for x in cursor.fetchall()]
    for t in t_ids:
        url = "http://rest.ensembl.org/overlap/id/" + t + "?feature=exon"
    

In [18]:
exons1 = ((1,10), (20,30), (40, 50), (60,70))
exons2 = ((5,10), (20,30), (40,50), (60,70))
exons3 = ((1,10), (20,30), (40, 50), (60,65))
exons4 = ((1,10), (15,30), (40, 50), (60,70))
exons5 = ((1,10), (20,30), (40, 50), (60,70))

exon_list = [exons1, exons2, exons3, exons4]
exon_list = list(set(exon_list))

splice_list = []

for e in exon_list:
    for e in exon_list:
        
        
        
        

4


In [21]:
import requests, json
cursor.execute('''SELECT chrom, start, end FROM gene_features''')
l = cursor.fetchone()
loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
headers = { "Content-Type" : "application/json"}
r = requests.get(url, headers=headers).json()

In [22]:
r

[{'binding_matrix': 'MA0093.2',
  'end': 203795517,
  'feature_type': 'motif',
  'motif_feature_type': 'USF1',
  'score': 9.421,
  'seq_region_name': '1',
  'start': 203795507,
  'strand': -1},
 {'binding_matrix': 'MA0093.2',
  'end': 203795518,
  'feature_type': 'motif',
  'motif_feature_type': 'USF1',
  'score': 11.608,
  'seq_region_name': '1',
  'start': 203795508,
  'strand': 1},
 {'binding_matrix': 'MA0281.1',
  'end': 203795516,
  'feature_type': 'motif',
  'motif_feature_type': 'USF1',
  'score': 9.032,
  'seq_region_name': '1',
  'start': 203795509,
  'strand': -1},
 {'binding_matrix': 'MA0281.1',
  'end': 203795516,
  'feature_type': 'motif',
  'motif_feature_type': 'USF1',
  'score': 9.032,
  'seq_region_name': '1',
  'start': 203795509,
  'strand': 1},
 {'binding_matrix': 'MA0095.2',
  'end': 203795542,
  'feature_type': 'motif',
  'motif_feature_type': 'Yy1',
  'score': 10.902,
  'seq_region_name': '1',
  'start': 203795531,
  'strand': -1},
 {'binding_matrix': 'MA0095.2',

In [47]:
cursor.execute('''UPDATE gene_features SET strand = ?''', (None,))
import time
cursor.execute('''SELECT id FROM gene_features''')
ids = [x[0] for x in cursor.fetchall()]
count = 0
t = time.time()
for i in ids:
    url = "http://rest.ensembl.org/overlap/id/" + i + "?feature=gene"
    headers = { "Content-Type" : "application/json"}
    r = requests.get(url, headers=headers).json()
    r = list(filter(lambda x: x['gene_id'] == i, r))
    s = r[0]['strand']
    cursor.execute('''UPDATE gene_features SET strand = ? WHERE id == ?''', (s,i))
    count += 1
    if count % 500 == 0:
        print(count, 'done:', (time.time()-t)/count, 'per gene')
db.commit()

500 done: 0.09891739797592163 per gene
1000 done: 0.09611216711997986 per gene
1500 done: 0.09632739337285359 per gene
2000 done: 0.09611967849731445 per gene
2500 done: 0.0953824215888977 per gene
3000 done: 0.09467443100611368 per gene
3500 done: 0.09394787284306118 per gene
4000 done: 0.09381823974847793 per gene
4500 done: 0.09417672644721137 per gene
5000 done: 0.09381783761978149 per gene
5500 done: 0.0935375654480674 per gene
6000 done: 0.09307543901602428 per gene
6500 done: 0.09281482846920307 per gene
7000 done: 0.09244800271306719 per gene
7500 done: 0.09228312892913819 per gene
8000 done: 0.09256074926257134 per gene
8500 done: 0.09250784046509687 per gene
9000 done: 0.0923375665611691 per gene
9500 done: 0.09214275894666973 per gene
10000 done: 0.09194323880672454 per gene
10500 done: 0.0918644430523827 per gene
11000 done: 0.09174451381509954 per gene
11500 done: 0.09163947366631549 per gene
12000 done: 0.09171679524580638 per gene
12500 done: 0.091671354637146 per gene
1

In [7]:
#get regulatory motifs within a gene and 5kb each side
# cursor.execute('''ALTER TABLE gene_features ADD COLUMN motif_number_1k INTEGER''')

chrom_list = list(range(1:23))
chrom_list = chrom_list.extend(['X', 'Y'])

#Maybe cycle through each chromosome and each strand
    #can't specify strand in url, will need to filter
for c in chrom_list:
    c = (c,-1)
import requests, json, time
cursor.execute('''SELECT chrom, start, end FROM gene_features''')
gene_locs = cursor.fetchall()
count = 0
counter = 0
count_t = time.time()
t = time.time()

for i in ids:
    count += 1
    counter += 1
    if time.time()-t <= 1 and count >= 15:
        
        time.sleep(0.5)
        t = time.time()
        count = 0
        
        cursor.execute('''SELECT chrom, start, end FROM gene_features WHERE id ==?''', (i,))
        l = cursor.fetchall([0])
        test_loc1 = l[1]
        loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
        
        url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
        headers = { "Content-Type" : "application/json"}
        r = requests.get(url, headers=headers).json()
        
        if r != []:
            cursor.execute('''UPDATE gene_features SET motif_number_1k =? WHERE id == ?''', (len(r), i))
        else:
            cursor.execute('''UPDATE gene_features SET motif_number_1k = 0 WHERE id == ?''', (i,))
    elif time.time()-t > 1:
        
        t = time.time()
        count = 0
        
        cursor.execute('''SELECT chrom, start, end FROM gene_features WHERE id ==?''', (i,))
        l = cursor.fetchall()[0]
        loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
        
        url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
        headers = { "Content-Type" : "application/json"}
        r = requests.get(url, headers=headers).json()
        
        if r != []:
            cursor.execute('''UPDATE gene_features SET motif_number_1k =? WHERE id == ?''', (len(r), i))
        else:
            cursor.execute('''UPDATE gene_features SET motif_number_1k = 0 WHERE id == ?''', (i,))
    else:
        
        cursor.execute('''SELECT chrom, start, end FROM gene_features WHERE id ==?''', (i,))
        l = cursor.fetchall()[0]
        loc = str(l[0]) + ':' + str(l[1]-1000) + '-' + str(l[2]+1000)
        
        url = "http://rest.ensembl.org/overlap/region/human/" + loc + "?feature=motif"
        headers = { "Content-Type" : "application/json"}
        r = requests.get(url, headers=headers).json()
        
        if r != []:
            cursor.execute('''UPDATE gene_features SET motif_number_1k =? WHERE id == ?''', (len(r), i))
        else:
            cursor.execute('''UPDATE gene_features SET motif_number_1k = 0 WHERE id == ?''', (i,))
    if counter% 200 == 0:
        print(counter, time.time()-count_t, 'seconds since started')

200 28.010648012161255 seconds since started
400 89.42845702171326 seconds since started
600 116.26174211502075 seconds since started
800 140.57208013534546 seconds since started
1000 164.35743021965027 seconds since started
1200 187.1804871559143 seconds since started
1400 210.7606041431427 seconds since started
1600 234.3334801197052 seconds since started
1800 256.11987018585205 seconds since started
2000 278.8302540779114 seconds since started
2200 302.30669713020325 seconds since started
2400 324.29499101638794 seconds since started
2600 347.12424516677856 seconds since started
2800 369.93838000297546 seconds since started
3000 392.0380640029907 seconds since started
3200 413.6419150829315 seconds since started
3400 436.07618713378906 seconds since started
3600 457.31153106689453 seconds since started
3800 478.27139019966125 seconds since started
4000 500.571093082428 seconds since started
4200 522.7625930309296 seconds since started
4400 544.4261062145233 seconds since started
460